In [1]:
import duckdb
import pandas as pd
from nba_api.stats.endpoints import commonallplayers, leaguedashplayerstats

In [5]:
# Pull players into a dataframe

players = commonallplayers.CommonAllPlayers(is_only_current_season=0)

df_players = players.get_data_frames()[0]

In [3]:
# Pull league stats for 2024-2025 season

league_stats = leaguedashplayerstats.LeagueDashPlayerStats(season='2024-25', per_mode_detailed='PerGame')

df_league_stats =  league_stats.get_data_frames()[0]

df_league_stats.head(15)

,PLAYER_ID,PLAYER_NAME,NICKNAME,TEAM_ID,TEAM_ABBREVIATION,AGE,GP,W,L,W_PCT,...,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,NBA_FANTASY_PTS_RANK,DD2_RANK,TD3_RANK,WNBA_FANTASY_PTS_RANK
0,1630639,A.J. Lawson,A.J.,1610612761,TOR,24.0,26,14,12,0.538,...,330,201,247,207,219,300,292,159,44,281
1,1631260,AJ Green,AJ,1610612749,MIL,25.0,73,44,29,0.603,...,467,520,136,394,278,83,342,281,44,318
2,1642358,AJ Johnson,AJ,1610612764,WAS,20.0,29,8,21,0.276,...,449,120,239,321,273,522,332,281,44,328
3,203932,Aaron Gordon,Aaron,1610612743,DEN,29.0,51,33,18,0.647,...,290,84,277,89,98,26,127,134,44,124
4,1628988,Aaron Holiday,Aaron,1610612745,HOU,28.0,62,39,23,0.629,...,385,415,413,350,363,142,422,281,44,403
5,1630174,Aaron Nesmith,Aaron,1610612754,IND,25.0,45,29,16,0.644,...,231,182,52,169,147,53,193,281,44,180
6,1630598,Aaron Wiggins,Aaron,1610612760,OKC,26.0,76,62,14,0.816,...,327,170,349,326,146,14,187,159,44,177
7,1641745,Adam Flagler,Adam,1610612760,OKC,25.0,37,34,3,0.919,...,478,514,521,555,514,222,527,281,44,523
8,1641766,Adama Sanogo,Adama,1610612741,CHI,23.0,4,2,2,0.500,...,519,348,540,521,503,367,526,281,44,524
9,1641737,Adem Bona,Adem,1610612755,PHI,22.0,58,12,46,0.207,...,30,373,134,192,355,404,312,134,44,334
